# **Pf8 Data Access**

For a quick overview on the spatial and geographical distribution of the samples available, please visit our [Pf8 web-app](https://apps.malariagen.net/apps/pf8/).

## Default Access: no authentication required
Plasmodium falciparum version 8 (Pf8) data are stored and openly accessible from the Wellcome Sanger Institute's [S3 storage](https://pf8-release.cog.sanger.ac.uk/index.html). No authentication is required to access these data using the [malariagen_data](https://github.com/malariagen/malariagen-data-python) Python package's default settings.

## Google Cloud Access: how to authenticate
Some Pf8 data (sample metadata, reference genome, genome annotations, genotype information in zarr format, genetic distance matrix) are accessible via Google Cloud Storage. Access to Google Cloud requires users to register for authentication. For instructions on how to authenticate, please see [here](https://malariagen.github.io/parasite-data/cloud_data_access/parasite-cloud-data-access.html).

## Acessing Pf8 with the malariagen_data Python package
This notebook illustrates how to read data directly from the cloud (S3 or Google), **without having to first download any data locally**. This notebook can be run from any computer, but will work best when run from a compute node within Google Cloud, because it will be physically closer to the data and so data transfer is faster. For example, this notebook can be run via MyBinder or Google Colab which are free interactive computing service running in the cloud.

To launch this notebook in the cloud and run it for yourself, click the launch icon (shaped like a rocket) at the top of the page and select one of the cloud computing services available.



## Setup

Running this notebook requires some Python packages to be installed. These packages can be installed via pip or conda. E.g.:

In [1]:
!pip install -q --no-warn-conflicts malariagen_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.9/775.9 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 k

To make accessing these data more convenient, we’ve created the malariagen_data Python package, which is available from PyPI. This is experimental so please let us know if you find any bugs or have any suggestions.

Now import the packages we’ll need to use here.

In [2]:
import numpy as np
import dask
import dask.array as da
from dask.diagnostics.progress import ProgressBar
import allel
# silence some warnings
dask.config.set(**{'array.slicing.split_large_chunks': False})
import malariagen_data

To access the pf8 data stored on S3 (default option), requiring no authentication, use the following code:

In [3]:
pf8 = malariagen_data.Pf8()

To access the pf8 data stored on google cloud use the following code:

In [4]:
pf8_gcs = malariagen_data.Pf8("gs://pf8-release/")

## Metadata

We will continue using the data accessed from S3.

Data on the samples that were sequenced as part of this resource are available. It includes the time and place of collection, quality metrics, and accession numbers.

To see all the information available, load sample metadata into a pandas dataframe:

In [5]:
pf8_metadata = pf8.sample_metadata()

pf8_metadata.head()

,Sample,Study,Country,Admin level 1,Country latitude,Country longitude,Admin level 1 latitude,Admin level 1 longitude,Year,ENA,All samples same case,Population,% callable,QC pass,Exclusion reason,Sample type,Sample was in Pf7
0,FP0008-C,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR1081237,FP0008-C,AF-W,82.48,True,Analysis_set,gDNA,True
1,FP0009-C,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR1081238,FP0009-C,AF-W,88.95,True,Analysis_set,gDNA,True
2,FP0010-CW,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR2889621,FP0010-CW,AF-W,87.01,True,Analysis_set,sWGA,True
3,FP0011-CW,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR2889624,FP0011-CW,AF-W,86.95,True,Analysis_set,sWGA,True
4,FP0012-CW,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR2889627,FP0012-CW,AF-W,89.86,True,Analysis_set,sWGA,True


In [6]:
print("The data set has {} samples and {} fields".format(pf8_metadata.shape[0],pf8_metadata.shape[1]))

The data set has 33325 samples and 17 fields


We can explore each of the fields:

- The <font color='purple'>Sample</font> column gives the unique sample identifier used throughout all Pf8 analyses.


- The <font color='purple'>Study</font> refers to the partner study which collected the sample.


- The <font color='purple'>Country</font> & <font color='purple'>Admin level 1</font> describe the location where the sample was collected from.


- The <font color='purple'>Country latitude</font>, <font color='purple'>Country longitude</font>, <font color='purple'>Admin level 1 latitude</font> and <font color='purple'>Admin 1 longitude</font> contain the GADM coordinates for each country & administrative level 1.


- The <font color='purple'>Year</font> column gives the time of sample collection.


- The <font color='purple'>ENA</font> column gives the run accession(s) for the sequencing read data for each sample.


- The <font color='purple'>All samples same case</font> column identifies samples set collected from the same individual.


- The <font color='purple'>Population</font> column gives the population to which the sample has been assigned. The possible values are: Africa - West (AF-W), Africa-Central (AF-C), Africa - East (AF-E), Africa - Northeast (AF-NE), Asia - South - East (AS-S-E), Asia - South – Far East (AS-S-FE), Asia - Southeast - West (AS-SE-W), Asia - Southeast - East (AS-SE-E), Oceania - New Guinea (OC-NG), South America (SA).


- The <font color='purple'>% callable</font> column refers to the %  of the genome with coverage of at least 5 reads and less than 10% of reads with mapping quality 0.


- The <font color='purple'>QC pass</font> column defines whether the sample passed (True) or failed (False) QC.
    
    
- The <font color='purple'>Exclusion reason</font> describes the reason why the particular sample was excluded from the main analysis.
    
    
- The <font color='purple'>Sample type</font> column gives details on the DNA preparation method used
    
    
- The <font color='purple'>Sample was in Pf7</font> column defines whether the sample was included in the previous version of the data release (Pf7) or if it is new to Pf8.

The python package [Pandas](https://pandas.pydata.org/) can be used to explore and query the sample metadata in different ways. For example, here is a summary of the numbers of samples grouped by the country they were collected in:

In [7]:
pf8_metadata.groupby("Country").size()

,0
Country,
Bangladesh,1658
Benin,334
Burkina Faso,58
Cambodia,2282
Cameroon,294
Colombia,167
Côte d'Ivoire,71
Democratic Republic of the Congo,1549
Ethiopia,35


## Variant Calls

Two variant callset versions were created for Pf8 from all samples in the release:
- a **Full callset:** Contains details of 12,493,205 discovered variant genome positions. Variants are single nucleotide polymorphisms (SNPs), short insertion/deletions (indels), or a combination of SNPs and indels.
- a **SNP-only callset:** A subset of the full callset, with all indel variants removed, leaving only SNPs. There are 10,821,552 SNPs in this callset.

Data on variant calls, including the genomic positions, alleles, and genotypes, can be accessed as an xarray Dataset:

In [8]:
# Access the full callset
variant_dataset = pf8.variant_calls()
variant_dataset

<xarray.Dataset> Size: 7TB
Dimensions:              (variants: 12493205, alleles: 7, samples: 33325,
                          ploidy: 2)
Coordinates:
    variant_position     (variants) int32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_chrom        (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>
    sample_id            (samples) object 267kB dask.array<chunksize=(16663,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele       (variants, alleles) object 700MB dask.array<chunksize=(131072, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_is_snp       (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_numalt       (variants) int32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_CDS          (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 833GB dask.array<chunksize=(131072, 100, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 6TB dask.array<chunksize=(131072, 100, 7), meta=np.ndarray>

In [10]:
# Access the SNP-only callset
variant_dataset_snp_only = pf8.variant_calls("pf8-release/snp-only/zarr/")
variant_dataset_snp_only

<xarray.Dataset> Size: 31TB
Dimensions:                                        (variants: 12493205,
                                                    alleles: 7, samples: 33325,
                                                    ploidy: 2, genotypes: 3,
                                                    sb_statistics: 4,
                                                    alt_alleles: 6)
Coordinates:
    variant_position                               (variants) int32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_chrom                                  (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>
    sample_id                                      (samples) object 267kB dask.array<chunksize=(16663,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                sb_statistics, alt_alleles
Data variables: (12/90)
    variant_allele                                 (variants, alleles) object 700MB dask.array<chunksize=(131072, 1), meta=np.ndarray>
    variant_filter_pass                            (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_is_snp                                 (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_numalt                                 (variants) int32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_CDS                                    (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    call_genotype                                  (variants, samples, ploidy) int8 833GB dask.array<chunksize=(131072, 100, 2), meta=np.ndarray>
    ...                                             ...
    variant_ReadPosRankSum                         (variants) float32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_RegionType                             (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_SOR                                    (variants) float32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_VQSLOD                                 (variants) float32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_culprit                                (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_set                                    (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>

The default returns a basic set of data most commonly used for data analysis. However, for more complex analysis the full range of variables available in the zarr can be accessed by setting the extended flag to `True`, as shown below:

In [11]:
extended_variant_dataset = pf8.variant_calls(extended=True)
extended_variant_dataset

<xarray.Dataset> Size: 31TB
Dimensions:                                        (variants: 12493205,
                                                    alleles: 7, samples: 33325,
                                                    ploidy: 2, genotypes: 3,
                                                    sb_statistics: 4,
                                                    alt_alleles: 6)
Coordinates:
    variant_position                               (variants) int32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_chrom                                  (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>
    sample_id                                      (samples) object 267kB dask.array<chunksize=(16663,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                sb_statistics, alt_alleles
Data variables: (12/90)
    variant_allele                                 (variants, alleles) object 700MB dask.array<chunksize=(131072, 1), meta=np.ndarray>
    variant_filter_pass                            (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_is_snp                                 (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_numalt                                 (variants) int32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_CDS                                    (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    call_genotype                                  (variants, samples, ploidy) int8 833GB dask.array<chunksize=(131072, 100, 2), meta=np.ndarray>
    ...                                             ...
    variant_ReadPosRankSum                         (variants) float32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_RegionType                             (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_SOR                                    (variants) float32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_VQSLOD                                 (variants) float32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_culprit                                (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_set                                    (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>

Each of the elements in this xarray dataset is a [dask array](https://docs.dask.org/en/latest/array.html). The individual dask arrays can be accessed as follows, replacing the string with the variable you are looking for:

In [12]:
pos = variant_dataset["variant_position"].data
pos

dask.array<array, shape=(12493205,), dtype=int32, chunksize=(131072,), chunktype=numpy.ndarray>

## Genotypes

Genotypes for individual samples are available.

Genotypes are stored as a three-dimensional array, where:

* the first dimension corresponds to genomic positions,
* the second dimension is samples,
* the third dimension is ploidy (2).

Values coded as integers, where -1 represents a missing value, 0 represents the reference allele, and 1, 2, and 3 represent alternate alleles.

Variant genotypes can be accessed as dask arrays as shown below.

In [13]:
gt = variant_dataset["call_genotype"].data
gt

dask.array<array, shape=(12493205, 33325, 2), dtype=int8, chunksize=(131072, 100, 2), chunktype=numpy.ndarray>

Note that the columns of this array (second dimension) match the rows in the sample metadata. You can use this correspondance to apply further subsetting operations to the genotypes by querying the sample metadata. E.g.:

In [14]:
loc_colombia = pf8_metadata.eval("Country == 'Colombia'").values
print(f"found {np.count_nonzero(loc_colombia)} samples from Colombia")
variant_dataset_colombia = variant_dataset.isel(samples=loc_colombia)
variant_dataset_colombia

found 167 samples from Colombia


<xarray.Dataset> Size: 34GB
Dimensions:              (variants: 12493205, alleles: 7, samples: 167,
                          ploidy: 2)
Coordinates:
    variant_position     (variants) int32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_chrom        (variants) object 100MB dask.array<chunksize=(131072,), meta=np.ndarray>
    sample_id            (samples) object 1kB dask.array<chunksize=(167,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele       (variants, alleles) object 700MB dask.array<chunksize=(131072, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_is_snp       (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_numalt       (variants) int32 50MB dask.array<chunksize=(131072,), meta=np.ndarray>
    variant_CDS          (variants) bool 12MB dask.array<chunksize=(131072,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 4GB dask.array<chunksize=(131072, 99, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 29GB dask.array<chunksize=(131072, 99, 7), meta=np.ndarray>

The data on genomic variants can be loaded into memory as [numpy](https://numpy.org/doc/stable/reference/generated/numpy.array.html) arrays as shown in the following example, where we read genotypes for the first 5 SNPs and the first 3 samples:


In [15]:
g = gt[:5, :3, :].compute()
g

array([[[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]]], dtype=int8)

If you want to work with the genotype calls, you may find it convenient to use [scikit-allel](https://scikit-allel.readthedocs.io/en/stable/). E.g., the code below sets up a genotype array using the Colombian samples subset we created above.

In [16]:
# use the scikit-allel wrapper class for genotype calls
gt = allel.GenotypeDaskArray(variant_dataset_colombia["call_genotype"].data)
gt

<GenotypeDaskArray shape=(12493205, 167, 2) dtype=int8>

## Genome Annotations

Gene annotations provide information on which regions of the genome contain DNA sequences that encode genes, which are transcribed and spliced into messenger RNA (mRNA) and then translated to make proteins.

For convenience, we’ve added some functionality to the malariagen_data package for loading these gene annotations into a pandas data frame as shown below:

In [17]:
genome_features = pf8.genome_features()
genome_features

,contig,source,type,start,end,score,strand,phase,ID,Parent,Name
0,Pf3D7_13_v3,VEuPathDB,protein_coding_gene,624510,626292,NaN,+,NaN,PF3D7_1314600,NaN,LipL1
1,Pf3D7_13_v3,VEuPathDB,mRNA,624510,626292,NaN,+,NaN,PF3D7_1314600.1,PF3D7_1314600,NaN
2,Pf3D7_13_v3,VEuPathDB,exon,624510,626292,NaN,+,NaN,exon_PF3D7_1314600.1-E1,PF3D7_1314600.1,NaN
3,Pf3D7_13_v3,VEuPathDB,CDS,624785,626011,NaN,+,0.0,PF3D7_1314600.1-p1-CDS1,PF3D7_1314600.1,NaN
4,Pf3D7_13_v3,VEuPathDB,five_prime_UTR,624510,624784,NaN,+,NaN,utr_PF3D7_1314600.1_1,PF3D7_1314600.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
50065,Pf3D7_13_v3,VEuPathDB,five_prime_UTR,957496,958095,NaN,-,NaN,utr_PF3D7_1322600.1_2,PF3D7_1322600.1,NaN
50066,Pf3D7_02_v3,VEuPathDB,protein_coding_gene,701245,702894,NaN,+,NaN,PF3D7_0216900,NaN,NaN
50067,Pf3D7_02_v3,VEuPathDB,mRNA,701245,702894,NaN,+,NaN,PF3D7_0216900.1,PF3D7_0216900,NaN
50068,Pf3D7_02_v3,VEuPathDB,exon,701245,702894,NaN,+,NaN,exon_PF3D7_0216900.1-E1,PF3D7_0216900.1,NaN


The above loads a default set of attributes `"ID", "Parent", "Name", "alias"`. To access all features set `attributes` to `"*"`.

In [18]:
pf8.genome_features(attributes="*")

,contig,source,type,start,end,score,strand,phase,ID,Name,Note,Parent,description,gene_id,protein_source_id
0,Pf3D7_13_v3,VEuPathDB,protein_coding_gene,624510,626292,NaN,+,NaN,PF3D7_1314600,LipL1,NaN,NaN,lipoate-protein ligase 1,NaN,NaN
1,Pf3D7_13_v3,VEuPathDB,mRNA,624510,626292,NaN,+,NaN,PF3D7_1314600.1,NaN,2.3.1.181 (Lipoyl(octanoyl) transferase);2.7.7...,PF3D7_1314600,lipoate-protein ligase 1,NaN,NaN
2,Pf3D7_13_v3,VEuPathDB,exon,624510,626292,NaN,+,NaN,exon_PF3D7_1314600.1-E1,NaN,NaN,PF3D7_1314600.1,NaN,PF3D7_1314600,NaN
3,Pf3D7_13_v3,VEuPathDB,CDS,624785,626011,NaN,+,0.0,PF3D7_1314600.1-p1-CDS1,NaN,NaN,PF3D7_1314600.1,NaN,PF3D7_1314600,PF3D7_1314600.1-p1
4,Pf3D7_13_v3,VEuPathDB,five_prime_UTR,624510,624784,NaN,+,NaN,utr_PF3D7_1314600.1_1,NaN,NaN,PF3D7_1314600.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50065,Pf3D7_13_v3,VEuPathDB,five_prime_UTR,957496,958095,NaN,-,NaN,utr_PF3D7_1322600.1_2,NaN,NaN,PF3D7_1322600.1,NaN,NaN,NaN
50066,Pf3D7_02_v3,VEuPathDB,protein_coding_gene,701245,702894,NaN,+,NaN,PF3D7_0216900,NaN,NaN,NaN,"conserved Plasmodium protein, unknown function",NaN,NaN
50067,Pf3D7_02_v3,VEuPathDB,mRNA,701245,702894,NaN,+,NaN,PF3D7_0216900.1,NaN,NaN,PF3D7_0216900,"conserved Plasmodium protein, unknown function",NaN,NaN
50068,Pf3D7_02_v3,VEuPathDB,exon,701245,702894,NaN,+,NaN,exon_PF3D7_0216900.1-E1,NaN,NaN,PF3D7_0216900.1,NaN,PF3D7_0216900,NaN


Or to get a specific set of attributes specify them in a list

In [20]:
pf8.genome_features(attributes=['ID','Note','description'])

,contig,source,type,start,end,score,strand,phase,ID,Note,description
0,Pf3D7_13_v3,VEuPathDB,protein_coding_gene,624510,626292,NaN,+,NaN,PF3D7_1314600,NaN,lipoate-protein ligase 1
1,Pf3D7_13_v3,VEuPathDB,mRNA,624510,626292,NaN,+,NaN,PF3D7_1314600.1,2.3.1.181 (Lipoyl(octanoyl) transferase);2.7.7...,lipoate-protein ligase 1
2,Pf3D7_13_v3,VEuPathDB,exon,624510,626292,NaN,+,NaN,exon_PF3D7_1314600.1-E1,NaN,NaN
3,Pf3D7_13_v3,VEuPathDB,CDS,624785,626011,NaN,+,0.0,PF3D7_1314600.1-p1-CDS1,NaN,NaN
4,Pf3D7_13_v3,VEuPathDB,five_prime_UTR,624510,624784,NaN,+,NaN,utr_PF3D7_1314600.1_1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
50065,Pf3D7_13_v3,VEuPathDB,five_prime_UTR,957496,958095,NaN,-,NaN,utr_PF3D7_1322600.1_2,NaN,NaN
50066,Pf3D7_02_v3,VEuPathDB,protein_coding_gene,701245,702894,NaN,+,NaN,PF3D7_0216900,NaN,"conserved Plasmodium protein, unknown function"
50067,Pf3D7_02_v3,VEuPathDB,mRNA,701245,702894,NaN,+,NaN,PF3D7_0216900.1,NaN,"conserved Plasmodium protein, unknown function"
50068,Pf3D7_02_v3,VEuPathDB,exon,701245,702894,NaN,+,NaN,exon_PF3D7_0216900.1-E1,NaN,NaN


## Genome Reference

We mapped sequence reads for all samples against the P. falciparum 3D7 v3 reference genome.

For convenience, the reference genome sequence can be loaded as a dask array, e.g.:

In [21]:
ref = pf8.genome_sequence()
ref

dask.array<concatenate, shape=(23332839,), dtype=|S1, chunksize=(421914,), chunktype=numpy.ndarray>

This can be loaded as a numpy array using the following

In [22]:
ref.compute()

array([b'T', b'G', b'A', ..., b'A', b'T', b'A'], dtype='|S1')

The reference can also be subset by contig.

The set of contigs used can be accessed as follows:

In [23]:
pf8.contigs

['Pf3D7_01_v3',
 'Pf3D7_02_v3',
 'Pf3D7_03_v3',
 'Pf3D7_04_v3',
 'Pf3D7_05_v3',
 'Pf3D7_06_v3',
 'Pf3D7_07_v3',
 'Pf3D7_08_v3',
 'Pf3D7_09_v3',
 'Pf3D7_10_v3',
 'Pf3D7_11_v3',
 'Pf3D7_12_v3',
 'Pf3D7_13_v3',
 'Pf3D7_14_v3',
 'Pf3D7_API_v3',
 'Pf3D7_MIT_v3']

To load a single contig

In [24]:
pf8.genome_sequence(region='Pf3D7_01_v3')

dask.array<array, shape=(640851,), dtype=|S1, chunksize=(421914,), chunktype=numpy.ndarray>

To load multiple contigs specify them in a list. The data will be concatenated.

In [25]:
pf8.genome_sequence(region=['Pf3D7_07_v3','Pf3D7_02_v3','Pf3D7_03_v3'])

dask.array<concatenate, shape=(3460280,), dtype=|S1, chunksize=(421914,), chunktype=numpy.ndarray>

You can also specify a specific region of the contig.

In [26]:
pf8.genome_sequence(region=['Pf3D7_07_v3','Pf3D7_02_v3:15-20','Pf3D7_03_v3:40-50'])

dask.array<concatenate, shape=(1445224,), dtype=|S1, chunksize=(421914,), chunktype=numpy.ndarray>

If you know the gene name you would like to access, but aren't sure what the ID would be you can access this through the annotations. Below is an example for `CRT`.  

In [27]:
gene_name = str(genome_features.loc[genome_features.Name == 'CRT'].ID.values)
print(gene_name)

['PF3D7_0709000']


You can then enter this as the region

In [28]:
pf8.genome_sequence(region='PF3D7_0709000')

dask.array<getitem, shape=(3957,), dtype=|S1, chunksize=(3957,), chunktype=numpy.ndarray>